In [93]:
#import library
import os

import numpy as np
import pandas as pd
import geopandas as gpd
import reverse_geocode
from pycountry_convert import country_name_to_country_alpha2, convert_continent_code_to_continent_name, country_name_to_country_alpha3
from pycountry_convert.convert_country_alpha2_to_continent_code import country_alpha2_to_continent_code


In [152]:
#INPUT FILES
inc=pd.read_csv('./data/cases_20200211_china_JH.csv')
pop=pd.read_csv('./data/population_by_province_China.csv')
#traffic volumes and pop for cities with top 3 airports in each Chinease province
df=pd.read_csv('./data/province_airport_pop_traffic.csv')

In [153]:
inc_pop=inc.merge(pop,left_on='province',right_on='province')
inc_pop['inc']=inc_pop['cases']/inc_pop['population']
province_to_inc = pd.Series(inc_pop.inc.values, index = inc_pop.province).to_dict()

## Compute risk from i to $\alpha$  ( See SI)

## $r_{i\alpha}=\frac{e_{i}n_{i}A_{i\alpha}}{\sum_{j}e_{i}n_{i}}$ 

In [155]:
df['incidence_density'] = df['province'].map(province_to_inc) #  e_i
df['weight'] =  df['incidence_density'] * df['traffic'] #  e_i * n_i
df=df[['province', 'airport', 'traffic', 'incidence_density','weight']]

city_to_weight = pd.Series(df.weight.values, index = df.airport.values).to_dict() # maps airport to weight


df.head()

,province,airport,traffic,incidence_density,weight
0,Xinjiang,Karamay,590497.0,2.372336e-06,1.400857
1,Xinjiang,Urumqi,23027788.0,2.372336e-06,54.629654
2,Xinjiang,Kashi,2060100.0,2.372336e-06,4.887250
3,Tibet,Lhasa,4353948.0,3.134109e-07,1.364575
4,Tibet,Xigaze,81349.0,3.134109e-07,0.025496


In [156]:
# function to select African countries from epirsik data
def select_Africa(aa):
    aa['continent'] = aa['countries']
    aa['continent'].replace({"Cote d'Ivoire": 'Ivory Coast'}, inplace = True)
    aa['continent'] = aa['continent'].apply(lambda x: country_name_to_country_alpha2(x) )
    aa['continent'] = aa['continent'].apply(lambda x: country_alpha2_to_continent_code(x) )
    aa['continent'] = aa['continent'].apply(lambda x: convert_continent_code_to_continent_name(x))  
    aa = aa[aa['continent'] == "Africa"]
    return aa

In [157]:
airport_to_weight = pd.Series(df.weight.values, index = df.airport.values).to_dict() 

In [158]:
path = './data/Risk_from_epirisk/'
month = 'January'

tmp2 = pd.DataFrame()
for airpt in df.airport.unique():
    tmp = pd.read_csv( os.path.join( path, 'epirisk_{}_{}.csv'.format(airpt, month)))
    tmp["cord"]=list(zip(tmp["lat"],tmp["lng"]))
    coordinates = list(tmp["cord"])
    countries=[reverse_geocode.search(coordinates)[i]['country'] for i in range(len(coordinates))]
    tmp['countries']=countries
    
    # to normlize risk we considered all countries outside China
    tmp=tmp[tmp["countries"]!="China"]
    tmp=tmp[tmp["countries"]!="Taiwan"]
    tmp=tmp[tmp["countries"]!="Hong Kong"]
    tmp=tmp[tmp["countries"]!="Macau"]
    
    tmp['seed'] = airpt
    tot=tmp.risk.sum()
    tmp['epirisk_norm'] = tmp['risk']/tot
    tmp2=tmp2.append(tmp)
tmp2=tmp2[tmp2['countries']!='Timor-Leste'] # we deleted as it is not mapped in pycountry_convert libraries

In [159]:
tmp3 = select_Africa(tmp2)
tmp3=tmp3.merge(df,left_on="seed",right_on='airport',how='left')
tmp3["num"]=tmp3["weight"]*tmp3["epirisk_norm"] #  e_i * n_i * A_iα

In [163]:
den2 = 0
for city in tmp3.seed.unique():
    den2 += city_to_weight[city] #  summing e_i * n_i 

In [166]:
tmp3["value"]=tmp3["num"]/den2 # importation risk per each city
tmp3['countries'].replace({"Cote d'Ivoire": 'Ivory Coast'}, inplace = True)
tmp3['continent']=tmp3['countries'].apply(lambda x: country_name_to_country_alpha2(x) )
tmp3['continent']=tmp3['continent'].apply(lambda x: country_alpha2_to_continent_code(x))
tmp3['ISO3'] = tmp3['countries'].apply(lambda x: country_name_to_country_alpha3(x) )

In [167]:
### compute importation risk per each country

In [144]:
drisk = tmp3.groupby(['countries'])['num'].sum() / den2
drisk1 = tmp3.groupby(['countries'])['num'].sum() 

In [145]:
df=pd.DataFrame(drisk).reset_index()
df=df[['countries','num']]
df.columns=['countries','risk']
df['countries'].replace({"Cote d'Ivoire": 'Ivory Coast'}, inplace = True)
df['ISO3'] = df['countries'].apply(lambda x: country_name_to_country_alpha3(x) )

In [150]:
df.head()

,countries,risk,ISO3
0,Algeria,0.002342,DZA
1,Angola,0.000622,AGO
2,Botswana,0.000014,BWA
3,Cameroon,0.000194,CMR
4,Chad,0.000258,TCD
